<a href="https://colab.research.google.com/github/rjanow/Masterarbeit/blob/main/UV_Measurement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Hier werden die Rohdaten des BTS2048-UV-WP in eine nutzbare CSV-Datei geschrieben**

Dokumentenname: Import_UV_Measurement.ipynb

Es werden die OR0-Daten (NasaAmes-Format), die eigentlich für den Versand an das BFS gedacht sind umgewandelt und in eine CSV-Datei geschrieben.

In [ ]:
# import der benötigten Module
import os, sys
import json
import datetime
import pandas as pd
import numpy as np
import csv
import re

from scipy.io import netcdf
from datetime import timedelta

Zu Beginn muss die Google-Drive eingerichtet werden, in der die Messdaten (OR0-Dateien) gespeichert sind. Danach werden alle verfügbaren Unterordner aufgerufen. So wird geprüft, ob der Mount richtig funktioniert hat.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

drive_path = '/content/drive/MyDrive'
# Durchsuche den Google Drive-Pfad
for root, dirs, files in os.walk(drive_path):
    for dir in dirs:
        # Gib den Namen des Unterordners aus
        print(os.path.join(root, dir))

Mounted at /content/drive


Danach werden die einzelnen OR0-Dateien (NasaAmes Format) geladen und umgewandelt.

Das NasaAmes Format: https://espoarchive.nasa.gov/content/Ames_Format_Specification_v20

Die Messdaten sind unter dem FFI (File Format Index) 2005 gespeichert. Dieser Standard ist durch die Nasa nicht dokumentiert. Deshalb nachfolgend ein eigener Parser, der die Daten in eine nutzbare CSV umwandelt.

**Umwandeln einer Datei**

- **lines** = enthält die OR0-Datei in Zeilen aufgeteilt
- **header** = Enthält den Dateikopf
- **end_line_header** = enthält die Zeile an dem der Header endet

**Datei einlesen:**

In [ ]:
# Testweise Öffnen einer Datei um zu prüfen, ob die Verbindung zu Google Drive funktioniert
file_path = r'/content/drive/MyDrive/Colab Notebooks/NasaAmes_Messdaten/Data/SA220615.OR0'  # Dateipfad in der Google-Drive

with open(file_path, 'r') as file:
    # Inhalt der Datei lesen
    content = file.read()

In [ ]:
# Zeilen in eine Liste aufteilen
lines = content.split('\n')

**Header extrahieren:**

In [ ]:
# Funktion um den Dateiheader zu extrahieren
def extract_header(dataset):
    header = ""
    end_line_header = None

    for i, line in enumerate(dataset):
        header += line + "\n"
        if line.strip() == "Pyranometer: readout interval [secs]=5":
            end_line_header = i
            break


    return header, end_line_header

In [ ]:
header = extract_header(lines)

In [ ]:
print(header[0])

**Nach Datum in Header suchen:**

In [ ]:
def search_date(data):
    for line in data:
        print(line)
        if len(line.split()) == 3:
            return line.strip()
    return None

In [ ]:
result = search_date(header[0])

if result:
    print("Datum gefunden:", result)
else:
    print("Datum nicht gefunden.")

**Daten in einzelne Messungen aufteilen:**

In [ ]:
# Funktion um den Anfang einer einzelnen Messung zu finden
def split_dataset(dataset):
    data_packages = []
    current_package = []

    for i, line in enumerate(dataset):
        if line.strip().isdigit() and len(line.strip()) == 5:
            if i + 1 < len(dataset) and dataset[i + 1].strip().isdigit() and len(dataset[i + 1].strip()) in [2, 3]:
                if current_package:
                    data_packages.append(current_package)
                    current_package = []
            current_package.append(line)

    if current_package:
        data_packages.append(current_package)

    return data_packages

In [ ]:
data_packages = split_dataset(lines)

In [ ]:
for i, package in enumerate(data_packages):
    print("Datenpaket", i+1)
    print(package)

Datenpaket 1
['26460', '26580', '26700', '26820', '26940', '27060', '27180', '27300', '27420', '27540', '27660', '27780', '27900', '29700', '29820', '29940', '30060', '30180', '30300', '30420', '30540', '30660', '30780', '30900', '31020', '31140', '31260', '31380', '31500', '31620', '31740', '31860', '31980', '32100', '32220', '32340', '32460', '32580', '32700', '32820', '32940', '33060', '33180', '33300', '33420', '33540', '33660', '33780', '33900', '34020', '34140', '34260', '34380', '34500', '34620', '34740', '34860', '34980', '35100', '35220', '35340', '35460', '35580', '35700', '35820', '35940', '36060', '36180', '36300', '36420', '36540', '36660', '36780', '36900', '37020', '37140', '37260', '37380', '37500', '37620', '37740', '37860', '37980', '38100', '38340', '38460', '38580', '38700', '38820', '38940', '39060', '39180', '39300', '39420', '39540', '39660', '39780', '39900', '40020', '40140', '40260', '40380', '40500', '40620', '40740', '40860', '40980', '41100', '41220', '4134

In [ ]:
# Funktion um den Zeitstempel der Messung in eine Uhrzeit und ein Datum umzuwandeln
def seconds_to_time(time_seconds):
    data = []

    for sec in time_seconds:
        hours = int(sec) // 3600
        minutes = (int(sec) % 3600) // 60
        seconds = int(sec) % 60
        time = pd.to_datetime(f"{hours:02d}:{minutes:02d}:{seconds:02d}")
        data.append({'Stunden': hours, 'Minuten': minutes, 'Sekunden': seconds, 'Uhrzeit': time})

    df = pd.DataFrame(data)
    return df

In [ ]:
time_df = seconds_to_time(package)

# Ausgabe des DataFrames
print(time_df)

     Stunden  Minuten  Sekunden             Uhrzeit
0          7       21         0 2023-07-06 07:21:00
1          7       23         0 2023-07-06 07:23:00
2          7       25         0 2023-07-06 07:25:00
3          7       27         0 2023-07-06 07:27:00
4          7       29         0 2023-07-06 07:29:00
..       ...      ...       ...                 ...
319       18       49         0 2023-07-06 18:49:00
320       18       51         0 2023-07-06 18:51:00
321       18       53         0 2023-07-06 18:53:00
322       18       55         0 2023-07-06 18:55:00
323       18       57         0 2023-07-06 18:57:00

[324 rows x 4 columns]


**Umwandeln aller Messdaten**

In [ ]:
# Funktion um die OR0-Dateien einzulesen
def read_files_with_extension(folder_path, extension, min_file_size):
    file_contents = []

    for file_name in os.listdir(folder_path):
        if file_name.endswith(extension):
            file_path = os.path.join(folder_path, file_name)

            # Überprüfe die Dateigröße
            if os.path.getsize(file_path) >= min_file_size:
                with open(file_path, 'r') as file:
                    content = file.read()
                    file_contents.append(content)

    return file_contents

In [ ]:
folder_path = r"/content/drive/MyDrive/Colab Notebooks/NasaAmes_Messdaten/Data"
extension = '.OR0'
min_file_size = 100  # Mindestgröße der Datei in Bytes

result = read_files_with_extension(folder_path, extension, min_file_size)

Im nächsten Schritt werden die OR0-Dateien (NasaAmes Format) in eine CSV-Datei abgespeichert.


In [ ]:
def write_list_to_csv(result, file_path):
    with open(file_path, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(result)

In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/CSV_Messdaten/Messdaten.csv'  # Passe den Dateipfad entsprechend an

write_list_to_csv(result, file_path)